In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn

/var/folders/hz/8qhn2dwn2zs419cpd7l0w6q40000gn/T/ipykernel_91915/2877271140.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# KNN Classifier

In [7]:
# Import necessary libraries
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train, y_train)

# Step 5: Make predictions and evaluate
y_pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8067076923076923
Classification Report:
               precision    recall  f1-score   support

       False       0.85      0.93      0.89     27569
        True       0.22      0.11      0.15      4931

    accuracy                           0.81     32500
   macro avg       0.54      0.52      0.52     32500
weighted avg       0.76      0.81      0.78     32500



In [8]:
# Print class distribution
print(data["persoon_geslacht_vrouw"].value_counts())

persoon_geslacht_vrouw
0    66985
1    63015
Name: count, dtype: int64


# Neural Network 

In [3]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define a neural network model with no regularization
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for many epochs (which can lead to overfitting)
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1,validation_split=0.2)

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f"Training accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

Epoch 1/100
4875/4875 [==============================] - 5s 1ms/step - loss: 1.4221 - accuracy: 0.8177 - val_loss: 0.4479 - val_accuracy: 0.8484
Epoch 2/100
4875/4875 [==============================] - 5s 926us/step - loss: 0.3949 - accuracy: 0.8503 - val_loss: 0.3783 - val_accuracy: 0.8486
Epoch 3/100
4875/4875 [==============================] - 5s 984us/step - loss: 0.3731 - accuracy: 0.8510 - val_loss: 0.3463 - val_accuracy: 0.8487
Epoch 4/100
4875/4875 [==============================] - 5s 967us/step - loss: 0.3405 - accuracy: 0.8634 - val_loss: 0.3221 - val_accuracy: 0.8658
Epoch 5/100
4875/4875 [==============================] - 6s 1ms/step - loss: 0.3195 - accuracy: 0.8745 - val_loss: 0.3228 - val_accuracy: 0.8737
Epoch 6/100
4875/4875 [==============================] - 6s 1ms/step - loss: 0.2971 - accuracy: 0.8841 - val_loss: 0.3152 - val_accuracy: 0.8756
Epoch 7/100
4875/4875 [==============================] - 6s 1ms/step - loss: 0.2868 - accuracy: 0.8882 - val_loss: 0.2579 - 

In [ ]:
# Here we create a subset of the data that only has label checked = 1

# Let's load the dataset
data_onlychecked = data[data['checked'] == 1]
data_onlychecked.shape[0]
# Let's specify the features and the target
y_onlychecked = data_onlychecked['checked']
X_onlychecked = data_onlychecked.drop(['checked'], axis=1)
X_onlychecked = X_onlychecked.astype(np.float32)


In [15]:
check_loss, check_acc = model.evaluate(X_onlychecked, y_onlychecked)

print(f"Training accuracy: {check_acc}")


610/610 [==============================] - 0s 523us/step - loss: 0.7906 - accuracy: 0.6204
Training accuracy: 0.6203855872154236


# All labels 0
We will set all target variables to 0, making the overall accuracy pretty high considering the original dataset doesn't have that many 1's. It will be able to correctly classify a lot of examples, but not the ones with 1, because it simply doesn't know how to. 

In [2]:
data_AllLabels0 = pd.read_csv('data/investigation_train_large_checked.csv')
data_AllLabels0['checked'] = 0

In [3]:
# Let's specify the features and the target
y = data_AllLabels0['checked']
X = data_AllLabels0.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define a neural network model with no regularization
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for many epochs (which can lead to overfitting)
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1,validation_split=0.2)

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print(f"Training accuracy: {train_acc}")
print(f"Test accuracy: {test_acc}")

Epoch 1/20
4875/4875 [==============================] - 5s 995us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/20
4875/4875 [==============================] - 5s 937us/step - loss: 2.2010e-31 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/20
4875/4875 [==============================] - 5s 932us/step - loss: 2.2010e-31 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
4875/4875 [==============================] - 5s 982us/step - loss: 2.2010e-31 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
4875/4875 [==============================] - 5s 1ms/step - loss: 2.2010e-31 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
4875/4875 [==============================] - 5s 937us/step - loss: 2.2010e-31 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
4875/4875 [==============================] - 4s 881us/step - loss: 2.20

In [8]:
from sklearn.metrics import accuracy_score, classification_report

# Let's load the dataset
data = pd.read_csv('data/investigation_train_large_checked.csv')

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked'], axis=1)
X = X.astype(np.float32)

# Let's split the dataset into train and test
_, X_test, _, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
test_loss, test_acc = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

1016/1016 [==============================] - 0s 395us/step
Classification Report:
               precision    recall  f1-score   support

       False       0.85      1.00      0.92     27569
        True       0.00      0.00      0.00      4931

    accuracy                           0.85     32500
   macro avg       0.42      0.50      0.46     32500
weighted avg       0.72      0.85      0.78     32500



/Users/luca/Desktop/TUDelft/Y2/DSAIT4015_SETAIS/ai_testing_group_20/aitesting/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/luca/Desktop/TUDelft/Y2/DSAIT4015_SETAIS/ai_testing_group_20/aitesting/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/luca/Desktop/TUDelft/Y2/DSAIT4015_SETAIS/ai_testing_group_20/aitesting/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with